In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import is_datetime64_any_dtype

mainDf = pd.read_csv('.//customers-1000.csv')#df is the data frame containing the whole content of file
columns = np.array(mainDf.columns)#to know the number of colums in dataframe(csv) file
noOfColumns = len(columns)
noOfRows = len(mainDf.index)
print(f"Total columns = {noOfColumns}")
print(f"Total rows = {noOfRows}")
sortAccordingToDateTime = True
sortAccordingToComment = False
if sortAccordingToDateTime == True:
    mainDf = clusterDateTimeCol(mainDf) 
print(mainDf)
arrOfGroupNames = mainDf.index.get_level_values(0).unique()# this variable i containing the group names
# print(mainDf.loc[f'{arrOfGroupNames[1]}'])

Total columns = 12
Total rows = 1000
Index dateTime
Subscription Date date
Multicalled
                           Index      Customer Id First Name Last Name  \
Group              Sr No.                                                
Group of year 2020 0          40  BEBA4fDAA6C4adC     Rickey      Mays   
                   1         242  dEc837d5F13C1ed     Kendra    Waters   
                   2         617  F9AEc2F51C5EDaE     Parker     Russo   
                   3         148  EF5858dEe5f7649    Belinda  Ferguson   
                   4         952  a9feB7dCD2FFd4e      Larry       Key   
...                          ...              ...        ...       ...   
Group of year 2022 995       772  cDBe5cFdbB3Ae4F     Jasmin    Waters   
                   996       831  4A231C5DceB9739       Jack   Mendoza   
                   997       498  E040edB499A6132     Amanda    Santos   
                   998       198  a8FfE4fbd7910b9    Bethany   Barrera   
                   999   

C:\Users\tikes\AppData\Local\Temp\ipykernel_19208\4233717916.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_col = pd.to_datetime(fContent[col], errors='raise', dayfirst=True)
C:\Users\tikes\AppData\Local\Temp\ipykernel_19208\4233717916.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_col = pd.to_datetime(fContent[col], errors='raise', dayfirst=True)
C:\Users\tikes\AppData\Local\Temp\ipykernel_19208\4233717916.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_col = pd.to_datetime(fContent[col], errors='raise', dayfirst=True)
C:\Users\tikes\A

KeyError: '1'

In [60]:
    def clusterDateTimeCol(fContent):
        for col in fContent.columns:
            a = sort_if_year_column(fContent[col])
            if a == False:
                pass
            else:
                fContent[col] = a
                fContent = fContent.reset_index(drop=True)
                fContent = multiIndex(fContent,col)
            try:
                # Try to convert the column to datetime (auto format detection)
                #it is flexible for all type of date time format
                parsed_col = pd.to_datetime(fContent[col], errors='raise', dayfirst=True)
                # Check if all time values are 00:00:00 → meaning only dates
                if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()): #it is for date only 
                    print(f"{col} date")
                    fContent = clean_and_sort_date_column(fContent,col)
                    fContent = fContent.reset_index(drop=True)
                    fContent = multiIndex(fContent,col,)
                else:# it is for both date and time
                    print(f"{col} dateTime")
                    fContent = handle_datetime_column(df,col)
                    fContent = fContent.reset_index(drop=True)
                    fContent = multiIndex(fContent,col)
            except Exception:
                continue
        
        return fContent

In [61]:
#if Only year
def sort_if_year_column(column):
    try:
        # Step 1: Convert to string and strip whitespace
        cleaned = column.astype(str).str.strip()
        
        # Step 2: Check if all values match 4-digit pattern
        if cleaned.str.fullmatch(r'\d{4}').all():
            # Step 3: Convert to int and check valid year range
            years = cleaned.astype(int)
            if years.between(1800, 2050).all():
                print("✅ Year-only column detected. Sorted.")
                return years.sort_values(ignore_index=True)
            else:
               pass
        else:
            pass
    except Exception as e:
        print(f"Error: {e}")
    
    return False # Return as-is if not valid


In [62]:
#fuction which will sort the dates if the df has date containing columns
def clean_and_sort_date_column(dff, column_name, ascending=True):
    try:
         # Step 1: Convert to datetime (handles multiple formats)
        dff[column_name] = pd.to_datetime(dff[column_name], errors='coerce', dayfirst=True)
        
        # Step 2: Drop NaT (invalid formats)
        dff = dff.dropna(subset=[column_name])
        
        # Step 3: Sort the DataFrame by that column
        dff = dff.sort_values(by=column_name, ascending=ascending)

        # Optional: Format to clean date string (YYYY-MM-DD)
        dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')

        return dff
    
    except Exception as e:
        print(f"⚠️ Error while processing date column: {e}")
        return dff

In [63]:
# if df has column containing the date and time both
def handle_datetime_column(df, column_name, ascending):

    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name,ascending=ascending).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [64]:
#Fuction to get the years from teh date format
def extract_year(date_str):
    match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if match:
        return int(match.group(0))
    return None

# Function to extract all unique years from a pandas Series
def extract_all_years(series):
    print("extract_all_years called")
    setofyears = [None] * len(series)

    for i in range(len(series)):
        setofyears[i] = extract_year(str(series[i]).strip())

    # Filter None values and return sorted unique years
    return np.array(sorted(set(y for y in setofyears if y is not None)))


def get_last_indices_of_each_year(date_series):
    # Extract year
    years = date_series.dt.year
    
    # Create a DataFrame with index
    df = pd.DataFrame({'year': years})
    
    # Get last index of each year group
    last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()
    
    return last_indices


In [ ]:
#Group(MultiIndexing) accordind to the condition 
def multiIndex(dataFrame, colToCheck):
    print("Multicalled")
    
    # Step 1: Get last indices of each year
    yearsWithLastIndex = get_last_indices_of_each_year(pd.to_datetime(dataFrame[colToCheck]))
    nameOfGroups = list(yearsWithLastIndex.keys())
    last_indices = list(yearsWithLastIndex.values())
    
    # Step 2: Compute counts from last indices
    group_sizes = []
    prev = -1
    for idx in last_indices:
        group_sizes.append(idx - prev)
        prev = idx
    
    # Step 3: Create array per group
    objOfGroups = {
        f'key{i}': np.array([f'Group of year {year}'] * group_sizes[i])
        for i, year in enumerate(nameOfGroups)
    }

    # Step 4: Combine into one array
    outside = np.concatenate(list(objOfGroups.values()))
    
    # Step 5: Create inside index
    inside = np.arange(len(outside))
    
    multi_index = pd.MultiIndex.from_arrays([outside, inside], names=["Group", "Sr No."])
    
    dataFrame.set_index(multi_index,inplace=True)
    return dataFrame
